In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt


from glob import glob
from tifffile import imread
from tqdm import tqdm
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D
from stardist.plot import render_label
     
from tensorflow.keras.preprocessing import image
import os

from random import randint

: 

In [ ]:
np.random.seed(42)
lbl_cmap = random_label_cmap()

: 

In [ ]:
y_path = '/Users/anshugusain/tf-nuclei/env/content/training_y_256/'
X_path = '/Users/anshugusain/tf-nuclei/env/content/training_X_256/'
HEIGHT = 256
WIDTH = 256
CHANNELS = 3


X = np.zeros((len(os.listdir(os.path.join(X_path))), HEIGHT, WIDTH,CHANNELS), dtype = np.float32)
y = np.zeros((len(os.listdir(os.path.join(y_path))), HEIGHT, WIDTH), dtype = np.uint8)

for x in range(0,len(os.listdir(os.path.join(y_path)))):

  training_image = image.load_img(X_path+ 'image_' + str(x) + '.png', target_size=(256, 256,3))
  training_image = np.asarray(training_image)
  n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1] 
  X[x] = training_image
    
    
  mask_image = image.load_img(y_path + 'image_' + str(x) + '.png', target_size=(256, 256), grayscale=True)
  mask_image_array = np.asarray(mask_image)
  y[x] = mask_image_array

  x+=1


X=X / 255


: 

In [ ]:
X[0].shape

: 

In [ ]:
plt.imshow(X[0])

: 

In [ ]:
fig, ax = plt.subplots(7,8, figsize=(16,16))
for i,(a,x) in enumerate(zip(ax.flat, X)):
  a.imshow(x)
  a.set_title(i)
  [a.axis('off') for a in ax.flat]
  plt.tight_layout()
     

: 

In [ ]:
model = StarDist2D(None, name='stardist_training', basedir='/Users/anshugusain/tf-nuclei')

: 

In [ ]:
img_num = randint(0, len(X)-1)
img = X[0]
#img = normalize(X[16], 1,99.8, axis=axis_norm)
labels, details = model.predict_instances(img)


plt.figure(figsize=(8,8))
plt.imshow(img)
plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
plt.axis('off');


: 

In [ ]:
#Plot 

def example(model, i, show_dist=True):
    img = X[i]
    labels, details = model.predict_instances(img)

    plt.figure(figsize=(13,10))
    img_show = img
    coord, points, prob = details['coord'], details['points'], details['prob']
    plt.subplot(121); plt.imshow(img_show); plt.axis('off')
    a = plt.axis()
    _draw_polygons(coord, points, prob, show_dist=show_dist)
    plt.axis(a)
    plt.subplot(122); plt.imshow(img_show); plt.axis('off')
    plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
    plt.tight_layout()
    plt.show()
    
    
example(model, 16)

: 

In [ ]:
# model.predict_instances(..., n_tiles = (2,4,4)) crop the input images in the respective number of tiles (also takes care of voerlap)
segmented_img, details_img = model.predict_instances(my_img, verbose=True)

: 

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
plt.subplot(1,2,1)
plt.imshow(my_img)
plt.axis("off")
plt.title("input image")

plt.subplot(1,2,2)
plt.imshow(render_label(segmented_img, img=my_img))
plt.axis("off")
plt.title("prediction")

: 

: 